This notebook will export data from the [Ford Multi-AV Dataset](https://avdata.ford.com/downloads/default.aspx).

In [ ]:
from __future__ import print_function, division

import os
import sys
import rosbag
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from ros_numpy import numpify
import sensor_msgs

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Re-record ROS bag with converted PointCloud2

- Clone the source code from Ford.

```bash
cd ~/Documents
mkdir -p FordAVData/src
cd FordAVData/src 
git clone git@github.com:xiaodongzhao/AVData.git 
cd .. && catkin_make && source devel/setup.bash
```

- run the Ford ROS node

```bash
roslaunch ford_demo demo.launch map_dir:=/media/xiaodong/Data/FordAVDataset/Map/2017-08-04-Map1 calibration_dir:=/media/xiaodong/Data/FordAVDataset/Calibration/V2
```

- start the node to record output messages

```bash
rosbag record -a -x=.*_scan$ -O /media/xiaodong/Data/FordAVDataset/exported.bag
```

- start the ROS node to convert data

```bash
roslaunch ford_demo multi_lidar_convert.launch
```

- Replay the ROS bag

```bash
rosbag play /media/xiaodong/Data/FordAVDataset/2017-08-04-V2-Log1.bag
```


In [ ]:
bag_file = "/media/xiaodong/Data/FordAVDataset/2017-08-04-V2-Log1_exported.bag"

In [ ]:
bag = rosbag.Bag(bag_file)
info = bag.get_type_and_topic_info()
info.topics

In [ ]:
export_topic_types = ["sensor_msgs/PointCloud2", "sensor_msgs/Imu", "geometry_msgs/PoseStamped", "sensor_msgs/NavSatFix"]
topics = [k for k, v in info.topics.iteritems() if v.msg_type in export_topic_types]

In [ ]:
time_limit = 5

In [ ]:
print("Extracting topics %s from %s..." % (topics, bag_file))
sys.stdout.flush()
start_time = None
msg_map = {k.strip("/"): [] for k in topics}
for topic, msg, t in tqdm(bag.read_messages(topics=topics)):
    key = topic.strip("/")
    msg_map[key].append(msg)

    if start_time is None:
        start_time = t.to_sec()
    if t.to_sec() - start_time >= time_limit:
        break

In [ ]:
print("Message number ")
{k: len(v) for k, v in msg_map.iteritems()}

## Export

In [ ]:
output_file = "/home/xiaodong/Dropbox/ShareFolders/FordAV_export/FordAV_export.h5"

In [ ]:
if os.path.exists(output_file):
    os.remove(output_file)

store = pd.HDFStore(output_file)


def get_time(msg):
    return msg.header.stamp.secs + msg.header.stamp.nsecs * 1e-9

### IMU

In [ ]:
imu_topic = "imu"

In [ ]:
def iter_imu(msg_list):
    for msg in msg_list:
        m = {"time": get_time(msg)}
        for var_name in ["orientation", "angular_velocity", "linear_acceleration"]:
            var = getattr(msg, var_name)
            for attr_name in ["x", "y", "z", "w"]:
                if hasattr(var, attr_name):
                    attr = getattr(var, attr_name)
                    m["%s_%s" % (var_name, attr_name)] = attr
        yield m

In [ ]:
imu_df = pd.DataFrame(iter_imu(msg_map[imu_topic]))
store.put(imu_topic, imu_df)

### GNSS

In [ ]:
gps_topic = "gps"

In [ ]:
def iter_gps(msg_list):
    for msg in msg_list:
        m = {"time": get_time(msg)}
        for var_name in ["latitude", "longitude", "altitude", "position_covariance_type"]:
            m["%s" % (var_name)] = getattr(msg, var_name)
        yield m

In [ ]:
gps_df = pd.DataFrame(iter_gps(msg_map[gps_topic]))
store.put(gps_topic, gps_df)

### LiDAR

In [ ]:
lidar_topics = [t for t in msg_map.keys() if "pointcloud" in t and len(msg_map[t]) > 0]
lidar_topics

In [ ]:
def iter_pt(msg_list):
    for i, msg in enumerate(msg_list):
        msg.__class__ = sensor_msgs.msg.PointCloud2
        arr = numpify(msg)
        df = pd.DataFrame(arr)
        yield get_time(msg), df

In [ ]:
for topic in lidar_topics:
    print(topic)
    for t, df in iter_pt(msg_map[topic]):
        store.put("%s/time_%s" % (topic, "%d" % (t * 1e9)), df)

### Pose

In [ ]:
pose_topics = [t for t in msg_map.keys() if "pose" in t and len(msg_map[t])>0]
pose_topics

In [ ]:
def iter_pose(msg_list):
    for i, msg in enumerate(msg_list):
        m = {"time": get_time(msg)}
        position = msg.pose.position
        orientation = msg.pose.orientation
        for attr_name in ["x","y", "z", "w"]:
            if hasattr(position, attr_name):
                m["%s_%s"%("position", attr_name)] = getattr(position, attr_name)
            if hasattr(orientation, attr_name):
                m["%s_%s"%("orientation", attr_name)] = getattr(orientation, attr_name)
        yield m

In [ ]:
for topic in pose_topics:
    df = pd.DataFrame(iter_pose(msg_map[topic]))
    store.put(topic, df)

### Finish

In [ ]:
with open("/home/xiaodong/Dropbox/ShareFolders/FordAV_export/info.txt", "w") as f:
    f.write("Note: for PointCloud, the key is time_nanoseconds\n")
    f.write(store.info())

In [ ]:
store.close()